In [1]:
#import sys
import requests
import time
import datetime
import time
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.dates as md
from pandas.io.json import json_normalize
import pandas as pd

In [157]:
endpoint_url = "https://api.whale-alert.io/v1/transactions?"
# Our free api-key
headers = {"X-WA-API-KEY" : "WZmlatDSHiZJjbb7JeLSub1fGjb9qwnd"}
# define the parameters of the request
# start_time: for the free version we can get at most 1hr of data, 
# min_value: the minimum USD value of the transactions. For the free api min_value has to be above 500,000
st_time = int(time.time() - 3590)
params = {
    "min_value" : 1000000,
    "start" : st_time 
    }

# make the request
rcomp = requests.get(endpoint_url, headers = headers, params = params)
raw_query = rcomp.json()
#raw_query has ['result', 'cursor', 'count', 'transactions']
raw_data = pd.DataFrame(raw_query["transactions"])
#processing: extract the type of wallet doing the transactions
from_owner = []
to_owner = []
for i in range(0,len(raw_data)):
    from_owner.append(raw_data.iloc[i]['from']['owner_type'])
    to_owner.append(raw_data.iloc[i]['to']['owner_type'])
fo = pd.DataFrame(from_owner)
to = pd.DataFrame(to_owner)
#Putting everything into a single table
summary = raw_data[["amount","amount_usd","blockchain","transaction_type"]]
summary.insert(loc=4, column = "from", value = fo )
summary.insert(loc=5, column = "to", value = to )

#separate btc with alts
tmp = summary.set_index("blockchain", drop = False)
alts = tmp.drop(index = 'bitcoin')

#set a higher cutoff for btc, as btc transaction are frequently occuring
cutoff = 5000000
btc = tmp.loc['bitcoin']
final_btc = pd.DataFrame(columns = ["amount", "amount_usd","blockchain", "from", "to"])
j = 0
for i in range(0,len(btc)):
    #print(btc.iloc[i]["amount"])
    if btc.iloc[i]["amount_usd"] > cutoff:
        final_btc.loc[j] = btc.iloc[i]
        j += 1

#crypto_data = pd.DataFrame(response)
#crypto_data.to_csv('/Users/apple/Desktop/kaiko_test/test.csv)

In [158]:
final_btc

,amount,amount_usd,blockchain,from,to
0,1378.5323,10117450.0,bitcoin,unknown,unknown
1,1374.2358,10059534.0,bitcoin,unknown,unknown
2,1364.9009,10001712.0,bitcoin,unknown,unknown
3,2541.9297,18584278.0,bitcoin,unknown,unknown
4,1343.0692,9833283.0,bitcoin,unknown,unknown
5,1317.4020,9627919.0,bitcoin,unknown,unknown
